# Lecture 05: Advanced SQL

In [6]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

If you haven't yet, load in the `imdb_lecture` and `stops_lecture` databases.

In [2]:
!mkdir data
!cp -v -r ../lec02/data .
!cp -v -r ../lec04/data/stops_lecture.sql data/.

'../lec02/data/imdb_lecture.zip' -> './data/imdb_lecture.zip'
'../lec02/data/imdb_lecture.sql' -> './data/imdb_lecture.sql'
'../lec02/data/.ipynb_checkpoints' -> './data/.ipynb_checkpoints'
'../lec04/data/stops_lecture.sql' -> 'data/./stops_lecture.sql'


In [3]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS imdb_lecture'
!psql -h localhost -c 'CREATE DATABASE imdb_lecture' 
!psql -h localhost -d imdb_lecture -f data/imdb_lecture.sql

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 8405
COPY 4043
COPY 4923
COPY 1223
COPY 820
COPY 2420


In [4]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS stops_lecture'
!psql -h localhost -c 'CREATE DATABASE stops_lecture' 
!psql -h localhost -d imdb_lecture -f data/stops_lecture.sql

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 84779
COPY 211
CREATE INDEX
CREATE INDEX


# Exercise #1: String manipulation

In [7]:
%load_ext sql

In [9]:
%sql postgresql://127.0.0.1:5432/stops_lecture

In [13]:
%%sql
SELECT 'Hello' || 'World',
       STRPOS('Hello', 'el'),
       SUBSTRING('Hello' FROM 2 FOR 3)
;

Running query in 'postgresql://127.0.0.1:5432/stops_lecture'

1 rows affected.

?column?,strpos,substring
HelloWorld,2,ell


In [14]:
%%sql
                                    -- pattern -- replace
SELECT REGEXP_REPLACE('Hannah Montana', '(.*) ', '\1') as ex1,
       REGEXP_REPLACE('Hannah Montana', '(.*) (.*)', '\2, \1') as ex2,
       REGEXP_REPLACE('Phone Number 510 642 3214', '[a-zA-Z ]', '') as ex3,
       REGEXP_REPLACE('Phone Number 510 642 3214', '[a-zA-Z ]', '', 'g') as ex4
;

Running query in 'postgresql://127.0.0.1:5432/stops_lecture'

1 rows affected.

ex1,ex2,ex3,ex4
HannahMontana,"Montana, Hannah",hone Number 510 642 3214,5106423214


# Exercise 2

In [15]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [16]:
%sql --close postgresql://127.0.0.1:5432/stops_lecture

In [17]:
%sql postgresql://127.0.0.1:5432/imdb_lecture

Connecting and switching to connection postgresql://127.0.0.1:5432/imdb_lecture

## Choice A

In [19]:
%%sql
SELECT *
FROM people
LIMIT 10;

Running query in 'postgresql://127.0.0.1:5432/imdb_lecture'

10 rows affected.

person_id,name,born,died
nm0384214,Dwayne Hill,None,None
nm0362443,Dave Hardman,1960,None
nm1560888,Rich Pryce-Jones,None,None
nm0006669,William Sadler,1950,None
nm1373094,Giada De Laurentiis,1970,None
nm7316782,Janine Hartmann,None,None
nm8671663,Tereza Taliánová,2005,None
nm10480297,Chris Heywood,None,None
nm10803545,Chengao Zhou,None,None
nm9849414,Mark Langley,None,None


In [18]:
%%sql
SELECT REGEXP_REPLACE(name, '(.*) (.*)', '\2') as lastname,
       REGEXP_REPLACE(name, '(.*) (.*)', '\1') as firstname
FROM people;

Running query in 'postgresql://127.0.0.1:5432/imdb_lecture'

1223 rows affected.

lastname,firstname
Hill,Dwayne
Hardman,Dave
Pryce-Jones,Rich
Sadler,William
Laurentiis,Giada De
Hartmann,Janine
Taliánová,Tereza
Heywood,Chris
Zhou,Chengao
Langley,Mark


## Choice B

In [20]:
%%sql
SELECT REGEXP_REPLACE(name, '(.*) (.*)', '\1') AS firstname,
       COUNT(*) AS countname
FROM people
GROUP BY firstname
ORDER BY countname desc;

Running query in 'postgresql://127.0.0.1:5432/imdb_lecture'

871 rows affected.

firstname,countname
Chris,79
David,18
Michael,16
Steve,10
Peter,9
John,9
Tom,8
Mark,8
Dan,8
Paul,6


## Choice C

In [25]:
%%sql
WITH firstnames AS (
    SELECT REGEXP_REPLACE(name, '(.*) .*', '\1') as firstname
    FROM people )
SELECT MAX(LENGTH(firstname))
FROM firstnames;

Running query in 'postgresql://127.0.0.1:5432/imdb_lecture'

1 rows affected.

max
20


What is this first name? Why doesn't the below query work?

In [ ]:
%%sql
WITH firstnames AS (
    SELECT REGEXP_REPLACE(name, '(.*) (.*)', '\1') as firstname
    FROM people )
SELECT MAX(LENGTH(firstname)), firstname
FROM firstnames;

In [31]:
%%sql
WITH firstnames AS (
    SELECT REGEXP_REPLACE(name, '(.*) (.*)', '\1') as firstname
    FROM people )
SELECT MAX(LENGTH(firstname)), firstname
FROM firstnames
GROUP BY firstname
ORDER BY firstname desc;

Running query in 'postgresql://127.0.0.1:5432/imdb_lecture'

1201 rows affected.

max,firstname
28,Juan Francisco PérezVillalba
25,Shyamashis R.Bhattacharya
24,José María GonzálezSinde
24,Margarida Mercês deMello
24,Friedrich KarlPraetorius
23,Jean-ChristianPetitfils
23,Linda VidettiFigueiredo
23,Michelle van derHeijden
22,Michael EldonLobsinger
22,Marissa NicoleTiamfook


Solution:

In [28]:
%%sql
WITH firstnames AS (
    SELECT REGEXP_REPLACE(name, '(.*) (.*)', '\1') as firstname
    FROM people )
SELECT MAX(LENGTH(firstname)), firstname
FROM firstnames
GROUP BY firstname
ORDER BY max DESC;

Running query in 'postgresql://127.0.0.1:5432/imdb_lecture'

871 rows affected.

max,firstname
20,Juan Francisco Pérez
19,José María González
19,Margarida Mercês de
16,Dadinha Monteiro
16,Sreemoolanagaram
16,Michelle van der
15,Jonathan Taylor
15,Deirdre Coleman
15,Nam Hoi Sap-Sam
15,Michael Richard


# Exercise \#3: Grouping

In [ ]:
%reload_ext sql

In [ ]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS stops_lecture'
!psql -h localhost -c 'CREATE DATABASE stops_lecture' 
!psql -h localhost -d stops_lecture -f data/stops_lecture.sql

In [ ]:
%sql postgresql://127.0.0.1:5432/stops_lecture

In [ ]:
%%sql
SELECT COUNT(*)
FROM stops;

In [ ]:
%%sql
SELECT DISTINCT COUNT(*)
FROM Zips;

In [ ]:
%%sql
SELECT COUNT(*)
FROM Stops
    INNER JOIN Zips
    ON Stops.location = Zips.location;

# Demo: Windows and Casting

In [ ]:
%reload_ext sql

In [ ]:
%sql --close postgresql://127.0.0.1:5432/stops_lecture

In [ ]:
%sql postgresql://127.0.0.1:5432/imdb_lecture

Make a temporary other table

In [ ]:
%%sql
DROP TABLE IF EXISTS actor_title;
CREATE TABLE actor_title AS (
    SELECT t1.primary_title AS title, t1.title_id, a1.name
    FROM titles t1, people a1, crew c1
    WHERE t1.title_id = c1.title_id AND a1.person_id = c1.person_id
);


In [ ]:
%%sql
SELECT *
FROM actor_title
TABLESAMPLE BERNOULLI (5);

Choice A

In [ ]:
%%sql
SELECT title_id, name, title,
       AVG(LENGTH(name)) OVER (PARTITION BY title)
         AS avg_name_length
FROM actor_title
WHERE title LIKE 'The %'
ORDER BY title;

Choice B

In [ ]:
%%sql
SELECT title_id, name, title,
       CAST(AVG(LENGTH(name)) OVER (PARTITION BY title)
            AS INTEGER)
         AS avg_name_length
FROM actor_title
WHERE title LIKE 'The %'
ORDER BY title;

Choice C

In [ ]:
%%sql
SELECT title_id, name, title,
       LENGTH(name),
       RANK() OVER (PARTITION BY title ORDER BY LENGTH(name) DESC)
         AS name_rank
FROM actor_title
WHERE title LIKE 'A %'
ORDER BY title, name_rank;